In [1]:
# Making a very simple code to test oculus-to-robot deployment on UR10e

# initial imports
from teleop_controls.oculus_reader.oculus_reader.reader import OculusReader
from teleop_controls.misc.subprocess_utils import run_multiprocessed_command
from teleop_controls.misc.subprocess_utils import run_threaded_command
from rtde_control import RTDEControlInterface as RTDEControl
from rtde_receive import RTDEReceiveInterface as RTDEReceive
from teleop_controls.misc.transformations import *
import time
import math
import pyrealsense2 as rs
import cv2
import numpy as np
# import jax
import robotiq_gripper
import PySpin
import matplotlib.pyplot as plt
# from PIL import Image
import datetime
from scipy.spatial.transform import Rotation as R
import os
import csv
# import math3d as m3d

# constants
ROBOT_IP = '192.168.0.110'
GRIPPER_PORT = 63352

# Parameters
rtde_frequency = 500.0
flags = RTDEControl.FLAG_VERBOSE | RTDEControl.FLAG_UPLOAD_SCRIPT
ur_cap_port = 50002

# ur_rtde realtime priorities
rt_receive_priority = 90
rt_control_priority = 85

# Initialize ur-rtde recieve and control commands
rtde_r = RTDEReceive(ROBOT_IP, rtde_frequency, [], True, False, rt_receive_priority)
rtde_c = RTDEControl(ROBOT_IP, rtde_frequency, flags, ur_cap_port, rt_control_priority)

# set robot joints start position
robot_startposition = (
    math.radians(252),
    math.radians(-77),
    math.radians(132),
    math.radians(161),
    math.radians(-67),
    math.radians(-22)
)

print("Start Position:", robot_startposition)

# Move robot to preset start position
rtde_c.moveJ(robot_startposition)

# initialize gripper
print("Creating gripper...")
gripper = robotiq_gripper.RobotiqGripper()
print("Connecting to gripper...")
gripper.connect(ROBOT_IP, GRIPPER_PORT)
print("Activating gripper...")
gripper.activate()

2025-03-25 14:00:00.015 | DEBUG    | robotiq_gripper:move_and_wait_for_pos:286 - gripper status: ObjectStatus.AT_DEST
2025-03-25 14:00:00.094 | DEBUG    | robotiq_gripper:move_and_wait_for_pos:286 - gripper status: ObjectStatus.MOVING


RTDEReceiveInterface: realtime kernel not found, consider using a realtime kernel for better performance.
Connected successfully to: 192.168.0.110 at 30004
RTDE synchronization started
RTDEControlInterface: realtime kernel not found, consider using a realtime kernel for better performance.
Connected successfully to: 192.168.0.110 at 30004
Waiting for RTDE data synchronization to start...
RTDE synchronization started
A script was running on the controller, killing it!
Start Position: (4.39822971502571, -1.3439035240356338, 2.303834612632515, 2.8099800957108707, -1.1693705988362009, -0.3839724354387525)
Creating gripper...
Connecting to gripper...
Activating gripper...


2025-03-25 14:00:02.096 | DEBUG    | robotiq_gripper:move_and_wait_for_pos:286 - gripper status: ObjectStatus.MOVING


Gripper auto-calibrated to [3, 228]


In [2]:
# initialize the realsense cameras
# first camera
# first camera is the one facing the user on the table
pipeline = rs.pipeline()
config = rs.config()
# specify camera serial number (this is the user camera)
config.enable_device('128422270567')
print(f"Device: Intel Realsense D405")
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 15)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 15)
pipeline.start(config)
    
# second camera
# second camera displays the robot's wrist camera view for better depth perception
pipeline_robot = rs.pipeline()
config_robot = rs.config()
config_robot.enable_device('218622276856')
print(f"Device: Intel Realsense D405 - Robot Wrist Camera")
config_robot.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 15)
config_robot.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 15)
pipeline_robot.start(config_robot)

# initial frame fetch for display
# first camera:
firstpov_frames = pipeline.wait_for_frames()
color_frame = firstpov_frames.get_color_frame()
depth_frame = firstpov_frames.get_depth_frame()

# second camera:
secondpov_frames = pipeline_robot.wait_for_frames()
secondpov_color_frame = secondpov_frames.get_color_frame()
secondpov_depth_frame = secondpov_frames.get_depth_frame()

# if not depth_frame or not color_frame:
#     continue                        
firstpov_frame = np.asanyarray(color_frame.get_data())
depth_image = np.asanyarray(depth_frame.get_data())

# if not secondpov_color_frame or not secondpov_depth_frame:
#     continue
secondpov_frame = np.asanyarray(secondpov_color_frame.get_data())

# reshape image frames to be consistent with octo data
firstpov_frame = cv2.resize(np.array(firstpov_frame),(256,256))
secondpov_frame = cv2.resize(np.array(secondpov_frame),(256,256))

Device: Intel Realsense D405
Device: Intel Realsense D405 - Robot Wrist Camera


In [3]:
# primary view - pointgrey 3rd person camera

print("Getting primary camera set up...")

# retrieve camera
result = True
serial = '13440209'

system = PySpin.System.GetInstance()
camera_list = system.GetCameras()
num_cams = camera_list.GetSize()

if num_cams == 0:
    camera_list.Clear()
    print('No cameras found.')

# start camera stream
cam = camera_list.GetBySerial(serial)
cam.Init()

nodemap = cam.GetNodeMap()
nodemap_tldevice = cam.GetTLDeviceNodeMap()
sNodemap = cam.GetTLStreamNodeMap()
cam.PixelFormat.SetValue(PySpin.PixelFormat_BayerRG8)
print (f"Pixel format set to: {cam.PixelFormat.GetCurrentEntry().GetSymbolic()}")

# set acquisition mode to continuous
# first need to set value of enumeration node
node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode("AcquisitionMode"))
if not PySpin.IsAvailable(node_acquisition_mode) or not PySpin.IsWritable(node_acquisition_mode):
    print ("Unable to set acquisition mode to continuous (enum retrieval). Aborting...")

# get entry node from enumeration node
node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName("Continuous")
if not PySpin.IsAvailable(node_acquisition_mode_continuous) or not PySpin.IsReadable(node_acquisition_mode_continuous):
    print ("Unable to set acquisition mode to continuous (entry retrieval). Aborting...")

# retrieve integer value from entry node
acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()

# set integer value from entry node as new value of enumeration node
node_acquisition_mode.SetIntValue(acquisition_mode_continuous)

cam.BeginAcquisition()

time.sleep(1) # allow some time for startup for camera

height = cam.Height()
width = cam.Width()

# create inital frames observation array
frames = cam.GetNextImage() # add a buffer to the frames

cv2_img = frames.GetData().reshape(height, width, 1)
cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BAYER_RG2RGB)
primary_img = cv2.resize(np.array(cv2_img),(256,256))

# show all camera views stacked together horizontally
stacked_viewframes = np.hstack((cv2.flip(firstpov_frame, 1), secondpov_frame, primary_img))
cv2.imshow("Realsense", stacked_viewframes)

Getting primary camera set up...
Pixel format set to: BayerRG8


In [4]:
# function for applying transformation on the frame of reference later
# may need to edit this depending on our robot vs Franka
def vec_to_reorder_mat(vec):
    X = np.zeros((len(vec), len(vec)))
    for i in range(X.shape[0]):
        ind = int(abs(vec[i])) - 1
        X[i, ind] = np.sign(vec[i])
    return X

#Additional conversion from Rmat to axis_angle in radians and vice-versa 
def euler_to_axis(euler, degrees=False):
    return R.from_euler("xyz", euler, degrees=degrees).as_rotvec()
    
def axis_to_euler(axis, degrees=False):
    return R.from_rotvec(axis).as_euler("xyz", degrees=degrees)

In [5]:
# class for vr policy; contains variables for calculating action
# borrowed some parts from DROID, with some modifications
class VRPolicy:
    def __init__(
        self,
        right_controller: bool = True,
        max_lin_vel: float = 1,
        max_rot_vel: float = 1,
        max_gripper_vel: float = 1,
        spatial_coeff: float = 1,
        pos_action_gain: float = 5,
        rot_action_gain: float = 2,
        gripper_action_gain: float = 3,
        rmat_reorder: list = [-2, -1, -3, 4],
    ):
        self.oculus_reader = OculusReader()
        self.vr_to_global_mat = np.eye(4)
        self.max_lin_vel = max_lin_vel
        self.max_rot_vel = max_rot_vel
        self.max_gripper_vel = max_gripper_vel
        self.spatial_coeff = spatial_coeff
        self.pos_action_gain = pos_action_gain
        self.rot_action_gain = rot_action_gain
        self.gripper_action_gain = gripper_action_gain
        self.global_to_env_mat = vec_to_reorder_mat(rmat_reorder)
        self.controller_id = "r" if right_controller else "l"
        self.reset_orientation = True
        self.reset_state()
        
        # for reading from the robot
        # self.ur

        # Start State Listening Thread #
        run_threaded_command(self._update_internal_state)
    
    # method that resets the state of the controller    
    def reset_state(self):
        self._state = {
            "poses": {},
            "buttons": {"A": False, "B": False, "X": False, "Y": False},
            "movement_enabled": False,
            "controller_on": True,
        }
        self.update_sensor = True
        self.reset_origin = True
        self.robot_origin = None
        self.vr_origin = None
        self.vr_state = None

    def _update_internal_state(self, num_wait_sec=5, hz=50):
        last_read_time = time.time()
        while True:
            # Regulate Read Frequency #
            time.sleep(1 / hz)

            # Read Controller
            time_since_read = time.time() - last_read_time
            poses, buttons = self.oculus_reader.get_transformations_and_buttons()
            self._state["controller_on"] = time_since_read < num_wait_sec
            if poses == {}:
                continue

            # Determine Control Pipeline #
            toggled = self._state["movement_enabled"] != buttons[self.controller_id.upper() + "G"]
            self.update_sensor = self.update_sensor or buttons[self.controller_id.upper() + "G"]
            self.reset_orientation = self.reset_orientation or buttons[self.controller_id.upper() + "J"]
            self.reset_origin = self.reset_origin or toggled

            # Save Info #
            self._state["poses"] = poses
            self._state["buttons"] = buttons
            self._state["movement_enabled"] = buttons[self.controller_id.upper() + "G"]
            self._state["controller_on"] = True
            last_read_time = time.time()

            # Update Definition Of "Forward" #
            stop_updating = self._state["buttons"][self.controller_id.upper() + "J"] or self._state["movement_enabled"]
            if self.reset_orientation:
                rot_mat = np.asarray(self._state["poses"][self.controller_id])
                if stop_updating:
                    self.reset_orientation = False
                # try to invert the rotation matrix, if not possible, then just use the identity matrix
                try:
                    rot_mat = np.linalg.inv(rot_mat)
                except:
                    print(f"exception for rot mat: {rot_mat}")
                    rot_mat = np.eye(4)
                    self.reset_orientation = True
                self.vr_to_global_mat = rot_mat

    def _process_reading(self):
        # the rotation matrix is a 4x4 matrix
        # need to first convert into quaternion by taking the upper left 3x3 matrix (rmat_to_quat)
        # the translational values are the last column (rightmost column)
        # take these values as the position values x, y, z
        rot_mat = np.asarray(self._state["poses"][self.controller_id])
        rot_mat = self.global_to_env_mat @ self.vr_to_global_mat @ rot_mat
        vr_pos = self.spatial_coeff * rot_mat[:3, 3] 
        vr_quat = rmat_to_quat(rot_mat[:3, :3])
        vr_gripper = self._state["buttons"]["RTr" if self.controller_id == "r" else "LTr"]

        self.vr_state = {"pos": vr_pos, "quat": vr_quat, "gripper": vr_gripper}

    # Try position control instead of velocity for now
    # need to have the IK for UR10e
    # def _limit_velocity(self, lin_vel, rot_vel, gripper_vel):
    #     """Scales down the linear and angular magnitudes of the action"""
    #     # basically: if the velocity exceeds the max, it turns it into normalized value
    #     # similar to unit vectors, works by multiplying the input vector by the quotient of the max vector/magnitude of input vector
    #     # otherwise, the input vector is not modified (magnitude is < max velocity)
    #     lin_vel_norm = np.linalg.norm(lin_vel)
    #     rot_vel_norm = np.linalg.norm(rot_vel)
    #     gripper_vel_norm = np.linalg.norm(gripper_vel)
    #     if lin_vel_norm > self.max_lin_vel:
    #         lin_vel = lin_vel * self.max_lin_vel / lin_vel_norm
    #     if rot_vel_norm > self.max_rot_vel:
    #         rot_vel = rot_vel * self.max_rot_vel / rot_vel_norm
    #     if gripper_vel_norm > self.max_gripper_vel:
    #         gripper_vel = gripper_vel * self.max_gripper_vel / gripper_vel_norm
    #     return lin_vel, rot_vel, gripper_vel

    def _calculate_action(self, state_dict, include_info=False):
        # Read Sensor #
        if self.update_sensor:
            self._process_reading()
            self.update_sensor = False

        # Read Observation
        robot_pos = np.array(state_dict["cartesian_position"][:3]) # first 3 values are x, y, z
        robot_euler = state_dict["cartesian_position"][3:] # last 3 vectors are xy, xz, yz
        robot_quat = euler_to_quat(robot_euler)
        robot_gripper = state_dict["gripper_position"]

        # Reset Origin On Release #
        if self.reset_origin:
            self.robot_origin = {"pos": robot_pos, "quat": robot_quat}
            self.vr_origin = {"pos": self.vr_state["pos"], "quat": self.vr_state["quat"]}
            self.reset_origin = False

        # Calculate Positional Action #
        robot_pos_offset = robot_pos - self.robot_origin["pos"]
        target_pos_offset = self.vr_state["pos"] - self.vr_origin["pos"]
        pos_action = target_pos_offset - robot_pos_offset # this is the action delta

        # Calculate Euler Action #
        robot_quat_offset = quat_diff(robot_quat, self.robot_origin["quat"])
        target_quat_offset = quat_diff(self.vr_state["quat"], self.vr_origin["quat"])
        quat_action = quat_diff(target_quat_offset, robot_quat_offset)
        euler_action = quat_to_euler(quat_action)

        # Calculate Gripper Action #
        # vr gripper is the boolean trigger instead of the numerical value
        # closed if true, open if false
        gripper_action = self.vr_state["gripper"]

        # # Calculate Desired Pose #
        target_pos = pos_action + robot_pos
        target_euler = add_angles(euler_action, robot_euler)
        target_cartesian = np.concatenate([target_pos, target_euler])
        target_gripper = self.vr_state["gripper"]

        # # Scale Appropriately #
        pos_action *= self.pos_action_gain
        euler_action *= self.rot_action_gain
        # gripper_action *= self.gripper_action_gain
        # lin_vel, rot_vel, gripper_vel = self._limit_velocity(pos_action, euler_action, gripper_action)
        
        # desired pose
        # this is absolute/direct pose
        # target_cartesian = np.concatenate([target_pos, target_euler])
        # target_gripper = self.vr_state["gripper"]
        
        # scale the pose
        # may need to modify these values for ur10e
        # target_pos *= self.pos_action_gain
        # target_euler *= self.rot_action_gain
        # gripper_action *= self.gripper_action_gain
        # decided not to scale gripper action since it is already between 0 and 1

        # calculate delta action
        # action = np.concatenate([target_pos, target_euler, [gripper_action]])
        action = np.concatenate([pos_action, euler_action, [gripper_action]])

        # Prepare Return Values #
        # these are the values obtained from the oculus reader; target_gripper is the controller gripper state
        # the actual executed actions are stored in action
        info_dict = {"target_cartesian_position": target_cartesian, "target_gripper_position": target_gripper}
        # action = np.concatenate([lin_vel, rot_vel, [gripper_vel]])
        # action = action.clip(-1, 1) # limits values to -1, 1

        # Return #
        if include_info:
            return action, info_dict
        else:
            return action

    # method is not in use right now, but can call it to return status
        # ends the policy if you click A or B; A = success, B = failed experiment
    def get_info(self):
        return {
            "success": self._state["buttons"]["A"] if self.controller_id == 'r' else self._state["buttons"]["X"],
            "failure": self._state["buttons"]["B"] if self.controller_id == 'r' else self._state["buttons"]["Y"],
            "movement_enabled": self._state["movement_enabled"],
            "controller_on": self._state["controller_on"],
        }

    def get_next_action(self, obs_dict, include_info=False):
        if self._state["poses"] == {}:
            action = np.zeros(7)
            if include_info:
                return action, {}
            else:
                return action # this is the delta (7 values)
        return self._calculate_action(obs_dict, include_info=include_info)

In [6]:
# # initialize the oculus reader
oculus_reader = OculusReader() # initialize oculusreader object
# controller_id = "r" # right controller; switch to "l" if using left controller

# # read controller
# last_read_time = time.time()
# num_wait_sec = 5
# # regulate read frequency #
# hz = 50
# time.sleep(1/hz)
# time_since_read = time.time() - last_read_time
# poses, buttons = oculus_reader.get_transformations_and_buttons()

In [7]:
# function to store the robot state in a obs_dict dictionary
def create_obs_dict(rtde_r, gripper):
    # return a dictionary with keys 'cartesian_position' and 'gripper_position'
    # where the cartesian position is the current tcp pose (x, y, z, rx, ry, rz)
    # and the gripper_position is a normalized value for the gripper status (original values are 0 to 255, we normalize to result in a range of 0 to 1)
    # octo takes the action with 1 as open and 0 as closed, but the gripper takes 255 as closed and 0 as open, so we need to flip by subtracting from 1 (later)
    # for vr policy, gripper is actually closed if 1 and open if 0; flip
    
    obs_dict = {
        'cartesian_position': rtde_r.getActualTCPPose(),
        'gripper_position': round(gripper.get_current_position()/255)
    }
    print(obs_dict)

    return obs_dict

In [8]:
# crude storage of robot pose in an array, which will be written to csv file
# store oculus reader output as well
# also store observation images in a directory:

# initialize robot pose array
# poses = [] # will store [x, y, z, rx - roll, ry - pitch, rz - yaw, GRIPPER]
# oculusPoses = [] # will store the poses from oculusreader
# oculusButtons = [] # will store the buttons from oculusreader

# # save observations as image strips later
# cameraObs = [] # for the default primary pointgrey camera
# cameraObsWrist = [] # for the realsense wrist camera (optional)

# initialize constants for naming
current_time = datetime.datetime.now()

# os.makedirs('outputs', exist_ok=True)

nameString = f'outputs/run_{current_time.year}-{current_time.month}-{current_time.day}_{current_time.hour}-{current_time.minute}-{current_time.second}/'

os.makedirs(nameString, exist_ok=True)


# create csv files
poses = open(nameString + 'robotPose.csv', 'w', newline='') # store the poses [x, y, z, rx, ry, rz]
poseWriter = csv.writer(poses)

oculusPoses = open(nameString + 'oculusPose.csv', 'w', newline='') # store poses from oculusreader
oculusPoseWriter = csv.writer(oculusPoses)

oculusButtons = open(nameString + 'oculusButtons.csv', 'w', newline='') # store button state from oculusreader
oculusButtonWriter = csv.writer(oculusButtons)

# paths to store image observations
primary_cameraPath = nameString + 'cameraPrimary/'
wrist_cameraPath = nameString + 'cameraWrist/'

os.makedirs(primary_cameraPath, exist_ok=True)
os.makedirs(wrist_cameraPath, exist_ok=True)

In [9]:
### Conversions ###
def quat_to_euler1(quat, degrees=False):
    euler = R.from_quat(quat).as_euler("xyz", degrees=degrees)
    return euler

def euler_to_quat1(euler, degrees=False):
    return R.from_euler("xyz", euler, degrees=degrees).as_quat()

# adding poses
def add_quats1(delta, source):
    result = R.from_quat(delta) * R.from_quat(source)
    return result.as_quat()

def add_poses_ur(delta, source):
    # linear addition for x, y, z
    # quaternion multiplication for roll, pitch, yaw
    lin_sum = np.array(delta[:3]) + np.array(source[:3])
    rot_sum = quat_to_euler1(add_quats1(euler_to_quat1(delta[3:6]), euler_to_quat1(source[3:6])))
    result = np.concatenate([lin_sum, rot_sum])
    return result

In [10]:
# TO DO:
    # test on robot
    # for documented csv values, maybe also record the timestamp
    # look into IK for velocity-based ur10e control
    
count = 0 # for indexing and naming

# create vrpolicy object
oculus_policy = VRPolicy(right_controller = True) # change to false if using left controller

try:
    while True:
        # inside of this while loop, continuously grab actions calculated from the oculus
        # terminates teleop process if the user presses ESC on the cv2 window
        
        # if escape key is pressed on frame window
        if cv2.waitKey(1) % 0xFF == 27:
            # run visualize actions
            # visualizeActions(totalActions, images)
            # flag = True
            cv2.destroyAllWindows()
            # close the pointgrey camera
            print('Closing camera and RTDE.')

            pipeline.stop()
            pipeline_robot.stop()
            cam.EndAcquisition()
            # deinitialize camera
            cam.DeInit()
            del cam
            camera_list.Clear()
            system.ReleaseInstance()
            rtde_c.stopScript()
            break
        
        # recording robot poses data:
        # get the current tcp pose (6 values) and append the gripper value
        # the gripper position is from 0 to 255, where 0 is open and 255 is closed (source: https://sdurobotics.gitlab.io/ur_rtde/_static/robotiq_gripper.py)
        # we want to take range btwn 0 and 1; just divide by 255 to normalize
        # to align with octo, 1 is open and 0 is closed so we subtract from 255 and round all values to be 0 or 1
        # poses.append(rtde_r.getActualTCPPose().append(round((255 - gripper.get_current_position())/255)))
        
        # record oculusreader poses and buttons
        # last_read_time = time.time()
        oc_pose, oc_button = oculus_reader.get_transformations_and_buttons()
        # oculusPoses.append(poses)
        # oculusButtons.append(buttons)
        
        # record states and observations in csv files
        # print(rtde_r.getActualTCPPose())
        # print(round((255 - gripper.get_current_position())/255))
        robotPoseRow = rtde_r.getActualTCPPose()
        # print(robotPoseRow)
        # append the gripper to the robot poses
        # the recorded observations are flipped such that 1 is open and 0 is closed, as required by octo
        # NOTE THAT THE OPPOSITE VALUE IS SENT TO THE VR POLICY (i.e. 1 is closed and 0 is open)
        robotPoseRow.append(round((255 - gripper.get_current_position())/255))
        # print(robotPoseRow)
        poseWriter.writerow(robotPoseRow)

        oculusPoseWriter.writerow(oc_pose)
        
        oculusButtonWriter.writerow(oc_button)
        
        # record camera observations
        cv2.imwrite(f'{primary_cameraPath}primary_{count}.png', primary_img)
        cv2.imwrite(f'{wrist_cameraPath}wrist_{count}.png', secondpov_frame)
        # cameraObs.append(primary_img[:, :, ::-1]) # convert from bgr to rgb
        # cameraObsWrist.append(secondpov_frame[:, :, ::-1]) # convert from bgr to rgb
        
        # release camera image
        frames.Release()
        
        # TO DO:
        # CALCULATE ROBOT ACTION
        # create observation dictionary
        obs_dict = create_obs_dict(rtde_r, gripper)
        
        # run get_next_action to calculate robot action based on oculus observation
        read_robot_action, info = (oculus_policy.get_next_action(obs_dict=obs_dict, include_info=True))
        # robot_action contains [x, y, z, rx, ry, rz, gripper (0-1)]
        actual_tcp_pose = rtde_r.getActualTCPPose()
        robot_action = add_poses_ur(read_robot_action, actual_tcp_pose)
        robot_action[:3] = euler_to_axis(robot_action[:3]) # convert to axis rotation
        
        # APPLY ROBOT ACTION
        print("Executing action: ", robot_action)
        tcp_pose = robot_action[:6].tolist() # first 6 values are the action
    
        gripper_position_val = robot_action[-1] # last value is the robot gripper from 0 to 1, where 0 is closed and 1 is open
        
        # robot pose update
        run_threaded_command(rtde_c.moveL(tcp_pose, 0.5, 0.5, True)) # moveL to target position, use non-blocking; velocity and acceleration = 0.5
        print("Movement complete. Proceeding to next action.")
        
        # gripper control
        if (gripper_position_val != gripper.get_current_position()): # close/open the gripper based on current state vs target state
            run_threaded_command(gripper.move_and_wait_for_pos(int(gripper_position_val * 255), 255, 255)) # either 255 (closed) or 0 (open)
        
        # UPDATE OBSERVATION CAMERAS AND OTHER VARIABLES
        # update cameras
        # first camera:
        firstpov_frames = pipeline.wait_for_frames()
        color_frame = firstpov_frames.get_color_frame()
        depth_frame = firstpov_frames.get_depth_frame()

        # second camera:
        secondpov_frames = pipeline_robot.wait_for_frames()
        secondpov_color_frame = secondpov_frames.get_color_frame()
        secondpov_depth_frame = secondpov_frames.get_depth_frame()

        if not depth_frame or not color_frame:
            continue                        
        firstpov_frame = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(depth_frame.get_data())

        if not secondpov_color_frame or not secondpov_depth_frame:
            continue
        secondpov_frame = np.asanyarray(secondpov_color_frame.get_data())

        # reshape image frames to be consistent with octo data
        firstpov_frame = cv2.resize(np.array(firstpov_frame),(256,256))
        secondpov_frame = cv2.resize(np.array(secondpov_frame),(256,256))
        
        # get frames from gige camera
        frames = cam.GetNextImage()
        # make sure all frames are complete; not corrupted
        while True:
            if(frames.IsIncomplete()):
                print (f"Image incomplete with image status {frames.GetImageStatus()}")
                frames.Release()
                frames = cam.GetNextImage()
            else: break
        
        cv2_img = frames.GetData().reshape(height, width, 1)
        cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BAYER_RG2RGB)
        primary_img = cv2.resize(np.array(cv2_img),(256,256))

        # show all camera views stacked together horizontally
        stacked_viewframes = np.hstack((cv2.flip(firstpov_frame, 1), secondpov_frame, primary_img))
        cv2.imshow("Realsense", stacked_viewframes)
        
        count +=1  # update counter index
              
except KeyboardInterrupt:
    print("Control Interrupted!")
    rtde_c.stopScript()
    
# stop all processes and close files
# close csv files
poses.close()
oculusPoses.close()
oculusButtons.close()

cv2.destroyAllWindows()
# close the pointgrey camera
print('Closing camera and RTDE.')
# close realsense cameras
pipeline.stop()
pipeline_robot.stop()
cam.EndAcquisition()
# deinitialize camera
cam.DeInit()
del cam
camera_list.Clear()
system.ReleaseInstance()
rtde_c.stopScript()

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'bool' object is not callable
2025-03-25 14:00:08.995 | DEBUG    | robotiq_gripper:move_and_wait_for_pos:286 - gripper status: ObjectStatus.MOVING


{'cartesian_position': [-0.04684945337483325, 0.8258093155817601, 0.2370971623935036, -2.1200010202489468, -0.2915490990017483, 0.039451202932464347], 'gripper_position': 0}
Executing action:  [-0.14194598  0.81675339  0.24277563 -2.11998553 -0.29168354  0.03955468]
Movement complete. Proceeding to next action.


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'tuple' object is not callable
Exception in thread Thread-9:
Traceback (most recent call last):
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **se

{'cartesian_position': [-0.06384069571078901, 0.8242383576173142, 0.2382019865743627, -2.1198894905866785, -0.2915816335255379, 0.03941605887450644], 'gripper_position': 0}
Executing action:  [-0.15839603  0.81315316  0.25088184 -2.11995267 -0.29166153  0.03947139]
Movement complete. Proceeding to next action.
{'cartesian_position': [-0.06603503563362442, 0.8240315885600001, 0.2381809943634639, -2.120255254669872, -0.2916080944547768, 0.039282580539729675], 'gripper_position': 0}
Executing action:  [-0.14991519  0.81505607  0.24719114 -2.11959742 -0.29139267  0.03951081]
Movement complete. Proceeding to next action.
{'cartesian_position': [-0.06717450334381225, 0.8239173299215439, 0.23839312475900387, -2.119918973025659, -0.2917276817239977, 0.03955096099150417], 'gripper_position': 0}
Executing action:  [-0.14538089  0.81607481  0.24453831 -2.11994672 -0.29129101  0.03925927]
Movement complete. Proceeding to next action.
{'cartesian_position': [-0.0680901625155692, 0.8237725350990013,

Exception in thread Thread-17:
Traceback (most recent call last):
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'bool' object is not callable
2025-03-25 14:00:09.416 | DEBUG    | robotiq_gripper:move_and_wait_for_pos:286 - gripper status: ObjectStatus.AT_DEST
Exception in thread Thread-18:
Traceback (most recent call last):
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/

...

2025-03-25 14:01:14.094 | DEBUG    | robotiq_gripper:move_and_wait_for_pos:286 - gripper status: ObjectStatus.AT_DEST
Exception in thread Thread-1648:
Traceback (most recent call last):
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'tuple' object is not callable
Exception in thread Thread-1649:
Traceback (most recent call last):
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  Fi

{'cartesian_position': [-0.19381511023301745, 0.8175359315930226, 0.21406390466215094, -2.394305862166613, -0.24642917317914867, 0.08447717755097782], 'gripper_position': 0}
Executing action:  [-0.19246864  0.81598316  0.21109247 -2.39541372 -0.24630161  0.08455254]
Movement complete. Proceeding to next action.
{'cartesian_position': [-0.1936714340974684, 0.817166017194869, 0.21342232355651575, -2.394817090419956, -0.24647904430658, 0.08453256465464094], 'gripper_position': 0}
Executing action:  [-0.19417535  0.81720436  0.21381248 -2.39486504 -0.24631753  0.0845351 ]
Movement complete. Proceeding to next action.
{'cartesian_position': [-0.19416988171787364, 0.8172143333739719, 0.21397064314865566, -2.3945484211113084, -0.24633972906967525, 0.08456719352325084], 'gripper_position': 0}
Executing action:  [-0.19136199  0.81739702  0.21090426 -2.39515096 -0.24641144  0.08446698]
Movement complete. Proceeding to next action.
{'cartesian_position': [-0.19313573818673993, 0.8171053112056444,

Exception in thread Thread-1655:
Traceback (most recent call last):
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 953, in run
2025-03-25 14:01:14.366 | DEBUG    | robotiq_gripper:move_and_wait_for_pos:286 - gripper status: ObjectStatus.AT_DEST
    self._target(*self._args, **self._kwargs)
TypeError: 'bool' object is not callable
Exception in thread Thread-1656:
Traceback (most recent call last):
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/media/Data/miniconda3/envs/teleop_env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  Fil

Movement complete. Proceeding to next action.
Closing camera and RTDE.
Closing camera and RTDE.


RuntimeError: stop() cannot be called before start()

In [11]:
rtde_c.disconnect()
rtde_r.disconnect()

RTDE - Socket disconnected
RTDE - Socket disconnected
